In [1]:
import pandas as pd
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import ssl
import nltk
import json
import os
from gensim import corpora, models
from pprint import pprint

In [2]:
np.random.seed(2018)

data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [3]:
path_to_json = '/Users/muskansinghal/Desktop/Reddit/'

json_files = []

for file in os.listdir(path_to_json):
    if file.endswith('.json'):
        json_files.append(path_to_json + file)
        
print(json_files)

['/Users/muskansinghal/Desktop/Reddit/Literature.json', '/Users/muskansinghal/Desktop/Reddit/Audience.json', '/Users/muskansinghal/Desktop/Reddit/Rehearsal.json', '/Users/muskansinghal/Desktop/Reddit/Valence Electrons.json', '/Users/muskansinghal/Desktop/Reddit/RNA.json', '/Users/muskansinghal/Desktop/Reddit/Imagination.json', '/Users/muskansinghal/Desktop/Reddit/Earthquakes.json', '/Users/muskansinghal/Desktop/Reddit/Erosion.json', '/Users/muskansinghal/Desktop/Reddit/Hormones.json', '/Users/muskansinghal/Desktop/Reddit/Organs.json', '/Users/muskansinghal/Desktop/Reddit/Line.json', '/Users/muskansinghal/Desktop/Reddit/Particle.json', '/Users/muskansinghal/Desktop/Reddit/Friction.json', '/Users/muskansinghal/Desktop/Reddit/Percentages.json', '/Users/muskansinghal/Desktop/Reddit/Division.json', '/Users/muskansinghal/Desktop/Reddit/Muscle.json', '/Users/muskansinghal/Desktop/Reddit/Population.json', '/Users/muskansinghal/Desktop/Reddit/Music.json', '/Users/muskansinghal/Desktop/Reddit/Nu

In [ ]:
#documents

In [4]:
final_data = {}
count = 0
for file in json_files:
    with open(file) as json_file:
        data = json.load(json_file)
        data = dict(data)
        for i in data:
            final_data[count]=i
            count+=1
            for j in data[i]:
                final_data[count]=j
                count+=1
                
documents=pd.DataFrame(final_data.items(),columns=['index','headline_text'])
documents['headline_text']=documents['headline_text'].astype('str')

KeyboardInterrupt: 

In [ ]:
# documents=pd.DataFrame(final_data.items(),columns=['index','headline_text'])

In [ ]:
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

# Wordnet is an large, freely and publicly available lexical database for the English language
# aiming to establish structured semantic relationships between words.
nltk.download('wordnet')

In [ ]:
# lemmatization considers the context and converts the word to its meaningful base form
# based on the context it’s used, identify the ‘part-of-speech’ (POS) tag for the word in that
# specific context and extract the appropriate lemma.
# j -> Adjective, v -> Verb, n -> Noun. r -> Adverb

stemmer = SnowballStemmer('english')

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos = 'v'))

In [ ]:
# gensim.utils.simple_preprocess - Convert a document into a list of tokens.
# This lowercases, tokenizes, de-accents (optional)
def preprocess(text):
    result = []

    for token in gensim.utils.simple_preprocess(text):

        # all stopwords are removed and words that have fewer than 3 characters are removed
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [ ]:
'''
#doc_sample = "The striped bats are hanging on their feet for best"

#print('original document: ')
#words = []
print(doc_sample)
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(WordNetLemmatizer().lemmatize('went', pos='v'))
print(preprocess(doc_sample))
'''
lemmatizer = WordNetLemmatizer()
#print(lemmatizer.lemmatize('bats'))
#print(nltk.pos_tag(nltk.word_tokenize(doc_sample)))



processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs)
dictionary

In [ ]:
# Create a dictionary from ‘processed_docs’ containing the number of times a word appears in the training set.
count = 0
for key, value in dictionary.iteritems():
    print(key, value)
    count += 1
    if count > 10:
        break

In [ ]:
#Filter out tokens that appear in
#less than 15 documents (absolute number) or
#more than 0.5 documents (fraction of total corpus size, not absolute number).
#after the above two steps, keep only the first 100000 most frequent tokens.
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=1000)

In [ ]:
#For each document we create a dictionary reporting how many
#words and how many times those words appear. 
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus

In [ ]:
#Preview Bag Of Words for our sample preprocessed document
bow_doc_431 = bow_corpus[431]
for i in range(len(bow_doc_431)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_431[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_431[i][1]))

In [ ]:
#Create tf-idf model object using models
#then apply transformation to the entire corpus
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
for doc in corpus_tfidf:
    pprint(doc)
    break

In [ ]:
for i in corpus_tfidf:
    print(i)

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

# For each topic, we will explore the words occuring in that topic and its relative weight.
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
processed_docs[4310]


In [ ]:
# running LDA using bag-of-words
print(documents.loc[431])
for index, score in sorted(lda_model[bow_corpus[431]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))

In [ ]:
# running LDA using tf-idf 
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

In [ ]:
unseen_document = 'The Pythagorean equation relates the sides of a right triangle in a simple way'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

In [ ]:
lda_model.print_topics(20,num_words=15)[:10]


In [ ]:
lda_model_tfidf.show_topics()